In [3]:
import cv2
import sys


In [4]:
def get_data(input_path):
    """Parse the data from annotation file

	Args:
		input_path: annotation file path

	Returns:
		all_data: list(filepath, width, height, list(bboxes))
		classes_count: dict{key:class_name, value:count_num}
			e.g. {'Car': 2383, 'Mobile phone': 1108, 'Person': 3745}
		class_mapping: dict{key:class_name, value: idx}
			e.g. {'Car': 0, 'Mobile phone': 1, 'Person': 2}
	"""
    found_bg = False
    all_imgs = {}

    classes_count = {}

    class_mapping = {}

    i = 1

    with open(input_path, 'r') as f:

        print('Parsing annotation files')

        for line in f:

            # Print process
            sys.stdout.write('\r' + 'idx=' + str(i))
            i += 1

            line_split = line.strip().split(',')

            # Make sure the info saved in annotation file matching the format (path_filename, x1, y1, x2, y2, class_name)
            # Note:
            #	One path_filename might has several classes (class_name)
            #	x1, y1, x2, y2 are the pixel value of the origial image, not the ratio value
            #	(x1, y1) top left coordinates; (x2, y2) bottom right coordinates
            #   x1,y1-------------------
            #	|						|
            #	|						|
            #	|						|
            #	|						|
            #	---------------------x2,y2

            (filename, class_name, x1, y1, x2, y2) = line_split

            if class_name not in classes_count:
                classes_count[class_name] = 1
            else:
                classes_count[class_name] += 1

            if class_name not in class_mapping:
                class_mapping[class_name] = len(class_mapping)

            if filename not in all_imgs:
                all_imgs[filename] = {}

                img = cv2.imread('Dataset/' + filename)
                (rows, cols) = img.shape[:2]
                all_imgs[filename]['filepath'] = filename
                all_imgs[filename]['width'] = cols
                all_imgs[filename]['height'] = rows
                all_imgs[filename]['bboxes'] = []

            all_imgs[filename]['bboxes'].append({
                                        'class_id': class_mapping[class_name],
                                        'x_center': (int(x1) + int(x2)) / 2.0 / cols,
                                        'y_center': (int(y1) + int(y2)) / 2.0 / rows,
                                        'width': (int(x2) - int(x1)) / cols,
                                        'height': (int(y2) - int(y1)) / rows})

        all_data = []
        for key in all_imgs:
            all_data.append(all_imgs[key])

        return all_data, classes_count, class_mapping


In [5]:
trainImages, classesCount, classMapping= get_data('data.csv')

Parsing annotation files
idx=4888

In [6]:
import yaml

In [15]:
def splitData(dataset,trainSize:float=0.8):
    splitIndex = int(trainSize * len(dataset))
    train:list=[]
    test:list=[]
    for i, file in enumerate(dataset):
        if i < splitIndex:
            train.append(file)
        else:
            test.append(file)
    return train, test

In [ ]:
def create_yolo_yaml(data_paths, class_mapping,class_count, output_path='data.yaml'):
    """ 
    Create a YOLO configuration YAML file.
    
    Args:
        data_paths: dict with keys 'train', 'val', 'test' (optional)
                    Each value should be a path to the images directory or .txt file
        class_mapping: dict{key:class_name, value: idx}
                    e.g. {'Car': 0, 'Mobile phone': 1, 'Person': 2}
        output_path: path to save the YAML file
    """
    # Sort class names by their index to maintain order
    sorted_classes = [class_name for class_name, idx in 
                     sorted(class_mapping.items(), key=lambda x: x[1])]
    
    # Build the YAML structure
    yaml_data = {
        'path': '.',  # Root directory path (relative to current working directory)
        'train': data_paths.get('train', ''),
        'test': data_paths.get('test', ''),
        'nc': class_count,  # Number of classes
        'names': sorted_classes  # Class names in order
    }
    
    # Write YAML file
    with open(output_path, 'w') as f:
        yaml.dump(yaml_data, f, default_flow_style=False, sort_keys=False)

    return yaml_data


In [ ]:
dataCombined:dict={}
dataCombined['train'], dataCombined['test'] = splitData(trainImages)
datasetYAMLformal = create_yolo_yaml(dataCombined,classMapping,classesCount)

YAML configuration saved to: data.yaml
Number of classes: 3
Class names: ['WBC', 'RBC', 'Platelets']


{'path': '.',
 'train': [{'filepath': 'BloodImage_00000.jpg',
   'width': 640,
   'height': 480,
   'bboxes': [{'class_id': 0,
     'x_center': 0.58671875,
     'y_center': 0.5760416666666667,
     'width': 0.3609375,
     'height': 0.41458333333333336},
    {'class_id': 1,
     'x_center': 0.2046875,
     'y_center': 0.803125,
     'width': 0.165625,
     'height': 0.20625},
    {'class_id': 1,
     'x_center': 0.18125,
     'y_center': 0.596875,
     'width': 0.165625,
     'height': 0.20625},
    {'class_id': 1,
     'x_center': 0.4171875,
     'y_center': 0.8572916666666667,
     'width': 0.165625,
     'height': 0.20625},
    {'class_id': 1,
     'x_center': 0.71875,
     'y_center': 0.8302083333333333,
     'width': 0.14375,
     'height': 0.19375},
    {'class_id': 1,
     'x_center': 0.93359375,
     'y_center': 0.8447916666666667,
     'width': 0.1328125,
     'height': 0.20625},
    {'class_id': 1,
     'x_center': 0.809375,
     'y_center': 0.9291666666666667,
     'width': 